<a href="https://colab.research.google.com/github/yogadarkknight/RockPaperScissors_image_analysis/blob/main/Copy_of_RockPaperScissors_image_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Import Library yang dibutuhkan
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import os,zipfile,shutil

In [ ]:
#download dataset
!wget --no-check-certificate \
  https://dicodingacademy.blob.core.windows.net/picodiploma/ml_pemula_academy/rockpaperscissors.zip \
  -O /tmp/rockpaperscissors.zip

--2021-05-16 11:27:06--  https://dicodingacademy.blob.core.windows.net/picodiploma/ml_pemula_academy/rockpaperscissors.zip
Resolving dicodingacademy.blob.core.windows.net (dicodingacademy.blob.core.windows.net)... 52.239.197.36
Connecting to dicodingacademy.blob.core.windows.net (dicodingacademy.blob.core.windows.net)|52.239.197.36|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 322873683 (308M) [application/zip]
Saving to: ‘/tmp/rockpaperscissors.zip’

rockpaperscissors.z  12%[=>                  ]  40.01M  10.2MB/s    eta 35s    ^C


In [ ]:
#cek version tensorflow
print(tf.__version__)

2.3.0


In [ ]:
#ekstrak dataset
local_zip = '/tmp/rockpaperscissors.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoc, logs={}):
    if(logs.get('accuracy') > 0.96):
      print("akurasi di atas 96 %, hentikan training")
      self.model.stop_training = True

callbacks = myCallback()

**Membuat direktori dan membagi dataset**

In [ ]:
base_dir = '/tmp/rockpaperscissors'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'val')
rock_dir = os.path.join(base_dir, 'rock')
paper_dir = os.path.join(base_dir, 'paper')
scissors_dir = os.path.join(base_dir, 'scissors')

In [ ]:
os.mkdir(train_dir)
os.mkdir(validation_dir)

FileExistsError: ignored

In [ ]:
train_rock = os.path.join(train_dir,'rock')
train_paper = os.path.join(train_dir, 'paper')
train_scissors = os.path.join(train_dir, 'scissors')
val_rock = os.path.join(validation_dir, 'rock')
val_paper = os.path.join(validation_dir, 'paper')
val_scissors = os.path.join(validation_dir, 'scissors')

In [ ]:
os.mkdir(train_rock)
os.mkdir(train_paper)
os.mkdir(train_scissors)
os.mkdir(val_rock)
os.mkdir(val_paper)
os.mkdir(val_scissors)

**Training**

In [ ]:
train_rock_dir, val_rock_dir = train_test_split(os.listdir(rock_dir), test_size = 0.40)
train_paper_dir, val_paper_dir = train_test_split(os.listdir(paper_dir), test_size = 0.40)
train_scissors_dir, val_scissors_dir = train_test_split(os.listdir(scissors_dir), test_size = 0.40)

NameError: ignored

In [ ]:
# membuat direktori rock pada direktori data training
train_rock_dir = os.path.join(train_dir, 'rock')

# membuat direktori paper pada direktori data training
train_paper_dir = os.path.join(train_dir, 'paper')

# membuat direktori scissors pada direktori data training
train_scissors_dir = os.path.join(train_dir, 'scissors')

# membuat direktori rock pada direktori data validasi
validation_rock_dir = os.path.join(validation_dir, 'rock')

# membuat direktori paper pada direktori data validasi
validation_paper_dir = os.path.join(validation_dir, 'paper')

# membuat direktori scissors pada direktori data validasi
validation_scissors_dir = os.path.join(validation_dir, 'scissors')

In [ ]:
train_datagen = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=20,
                    horizontal_flip=True,
                    shear_range = 0.2,
                    fill_mode = 'nearest')

test_datagen = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=20,
                    horizontal_flip=True,
                    shear_range = 0.2,
                    fill_mode = 'nearest')

In [ ]:
train_generator = train_datagen.flow_from_directory(
        train_dir,  # direktori data latih
        target_size=(150, 150),  # mengubah resolusi seluruh gambar menjadi 150x150 piksel
        batch_size=4,
        # karena kita merupakan masalah klasifikasi lebih 2 kelas maka menggunakan class_mode = 'categorical'
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        validation_dir, # direktori data validasi
        target_size=(150, 150), # mengubah resolusi seluruh gambar menjadi 150x150 piksel
        batch_size=4, # karena kita merupakan masalah klasifikasi lebih 2 kelas maka menggunakan class_mode = 'categorical'
        class_mode='categorical')

Found 4376 images belonging to 4 classes.
Found 2188 images belonging to 3 classes.


In [ ]:
train_data = train_generator
valid_data = validation_generator

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(2, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(34, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(34, activation='relu'),
    tf.keras.layers.Dense(1, activation='swish')
])

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=tf.optimizers.Adam(learning_rate=1e-4),
              metrics=['accuracy'])

In [ ]:
model.fit(
      train_generator,
      steps_per_epoch=5,  # berapa batch yang akan dieksekusi pada setiap epoch
      epochs=25,
      validation_data=validation_generator, # menampilkan akurasi pengujian data validasi
      validation_steps=6,  # erapa batch yang akan dieksekusi pada setiap epoch
      verbose=2)

Epoch 1/25
5/5 - 6s - loss: 1.1921e-07 - accuracy: 0.7500 - val_loss: 1.1921e-07 - val_accuracy: 0.6667
Epoch 2/25
5/5 - 4s - loss: 1.1921e-07 - accuracy: 0.7500 - val_loss: 1.1921e-07 - val_accuracy: 0.6667
Epoch 3/25
5/5 - 8s - loss: 1.1921e-07 - accuracy: 0.7500 - val_loss: 1.1921e-07 - val_accuracy: 0.6667
Epoch 4/25
5/5 - 8s - loss: 1.1921e-07 - accuracy: 0.7500 - val_loss: 1.1921e-07 - val_accuracy: 0.6667
Epoch 5/25
5/5 - 6s - loss: 1.1921e-07 - accuracy: 0.7500 - val_loss: 1.1921e-07 - val_accuracy: 0.6667
Epoch 6/25
5/5 - 5s - loss: 1.1921e-07 - accuracy: 0.7500 - val_loss: 1.1921e-07 - val_accuracy: 0.6667
Epoch 7/25
5/5 - 4s - loss: 1.1921e-07 - accuracy: 0.7500 - val_loss: 1.1921e-07 - val_accuracy: 0.6667
Epoch 8/25
5/5 - 5s - loss: 1.1921e-07 - accuracy: 0.7500 - val_loss: 1.1921e-07 - val_accuracy: 0.6667
Epoch 9/25
5/5 - 3s - loss: 1.1921e-07 - accuracy: 0.7500 - val_loss: 1.1921e-07 - val_accuracy: 0.6667
Epoch 10/25
5/5 - 4s - loss: 1.1921e-07 - accuracy: 0.7500 - val

In [ ]:
# Visualize training history
from keras.models import Sequential
from keras.layers import Dense
import matplotlib.pyplot as plt
import numpy

# split into input (X) and output (Y) variables
X = accuracy [:0,8]
Y = epochs [:,8]

history= model.fit(X,Y,
      train_generator,
      steps_per_epoch=10,  # berapa batch yang akan dieksekusi pada setiap epoch
      epochs=25,
      validation_data=validation_generator, # menampilkan akurasi pengujian data validasi
      validation_steps=8,  # erapa batch yang akan dieksekusi pada setiap epoch
      verbose=2)

# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

NameError: ignored

In [ ]:
# Visualize training history
from keras.models import Sequential
from keras.layers import Dense
import matplotlib.pyplot as plt
import numpy

# create model
model = Sequential()
model.add(Dense(12, input_dim=8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# Fit the model
history = model.fit(X, Y, validation_split=0.33, epochs=150, batch_size=10, verbose=0)


OSError: ignored

In [ ]:
import numpy as np
from google.colab import files
from keras.preprocessing import image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = fn
  img = image.load_img(path, target_size=(150,150))
  imgplot = plt.imshow(img)
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  image = np.vstack([x])
  classes = model.predict(image, batch_size=10)
  
  print(fn)
  if classes==0:
    print('rock')
  elif classes==1:
    print('paper')
  else:
    print('scissors')